## Product Quantizer算法

使用PQ量化的第一步就是需要训练好聚类模型并获得码本，使得具有对新输入数据进行编码的能力

**位数表示法：** 如果选择使用8位二进制数表示，那么对应的十进制数是2<sup>8</sup>=256

In [6]:
import faiss
import numpy as np
from numpy.testing.print_coercion_tables import print_new_cast_table

# 创建数据
dim = 32
data = np.random.rand(10000, dim).astype('float32')

# 训练码本（向量维度，子向量数量，子向量质心数(位数)）
# 质心数将使用输入位数组成的最大二进制数进行表示，比如:8->11111111(二进制)->255(十进制)
pq = faiss.ProductQuantizer(dim, 8, 8)
pq.train(data)  # 由于内部也存在聚类算法与码本，故需要进行训练创建


### 编码数据

在获得码本之后，我们对于新输入的数据就可以执行子向量分类与按照码本进行编码

In [7]:
origin_data = np.random.rand(1, dim)
coded_data = pq.compute_codes(origin_data)
print(f"原始数据为:{origin_data}")
print(f"编码后的数据为:{coded_data}")
print(f"编码后数据的形状是:{coded_data.shape}")

原始数据为:[[0.99294721 0.14272079 0.53898738 0.01825288 0.92797153 0.02983403
  0.8182503  0.20260697 0.06123151 0.26266258 0.24201474 0.76200225
  0.27224378 0.46651195 0.7787937  0.45588632 0.72188776 0.21171221
  0.85208513 0.18232851 0.9064073  0.07140375 0.00366112 0.52985508
  0.57079784 0.11919303 0.80378421 0.56111623 0.42250082 0.51048513
  0.80761346 0.08234399]]
编码后的数据为:[[158  55  45  72 110  12 180 213]]
编码后数据的形状是:(1, 8)


我们可以发现，在PQ编码之后，数据从32维度下降到了8，压缩程度达到了4倍！

### 解码数据

当获得了编码后的数据之后，我们自然也可以按照码本对数据进行解码，获得原来维度的数据<br/>
但由于我们是按照子向量区域内的聚合中心进行解码的，所以会产生误差

In [15]:
decoded_data = pq.decode(coded_data)
print(f"解码后的数据为:{decoded_data}")
diff = decoded_data - origin_data
num = origin_data.shape[0] * origin_data.shape[1]
print(f"均方误差为:{np.sum(pow(diff, 2)) / num}")

解码后的数据为:[[0.88122296 0.07628218 0.5539083  0.12453993 0.91809887 0.12915483
  0.742432   0.1521527  0.11769522 0.29498875 0.24967924 0.8984066
  0.37344372 0.49124646 0.62598866 0.4441901  0.65576273 0.09641423
  0.9086838  0.30697593 0.9042923  0.12815714 0.12485918 0.3911063
  0.5089358  0.14131011 0.65261054 0.6108575  0.43215296 0.654124
  0.8632609  0.11567549]]
均方误差为:0.007251823452862301


## 重要参数:pq.verbose=True
在FAISS中，verbose 是一个用于控制输出调试信息的参数，主要用于查看算法的内部运行状态，便于开发者调试和优化模型。
